# imports

In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from schlichtanders.mymatplotlib import Centre
from matplotlib.colors import LogNorm

/usr/local/lib/python2.7/dist-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
import os, platform, sys
from pprint import pprint

In [3]:
__file__ = os.path.realpath('__file__')
if platform.system() == "Windows":
    from schlichtanders.myos import replace_unc
    __file__ = replace_unc(__file__)
__path__ = os.path.dirname(__file__)
__parent__ = os.path.dirname(__path__)

In [4]:
from sqlalchemy import Column, Integer, Unicode, UnicodeText, String, PickleType, Float, Boolean
from sqlalchemy import create_engine, MetaData, Table
from sqlalchemy.orm import sessionmaker, Session, create_session
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.ext.automap import automap_base
import operator as op
from collections import defaultdict
import csv
import heapq
from copy import copy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
inf = float('inf')

In [5]:
import experiment_toy_models
import experiment_util
from schlichtanders.mycontextmanagers import ignored
from schlichtanders.myobjects import Namespace
from evaluation import get_modes
import evaluation as eva

gnumpy: failed to import cudamat. Using npmat instead. No GPU will be used.


In [6]:
import numpy as np
import theano

In [7]:
model_names = { # sorted by optimization type
    "ml": ['baselinedet'],
    "annealing": ['baseline', 'mixture', 'planarflow', 'planarflowdet', 'radialflow', 'radialflowdet'],
}
model_prefixes = reduce(op.add, model_names.values())

In [8]:
_model_prefixes = [p+"_" for p in model_prefixes]

# Hyperparameters
# ===============

Base = declarative_base()

class Hyper(Base):
    __tablename__ = "hyper"
    id = Column(Integer, primary_key=True)

    # hyper parameters:
    x_true = Column(Float)
    max_epochs_without_improvement = Column(Integer)
    logP_average_n = Column(Integer)
    errorrate_average_n = Column(Integer)
    minus_log_s1 = Column(Integer)
    minus_log_s2 = Column(Integer)
    batch_size = Column(Integer)

    n_normflows = Column(Integer)

    opt_identifier = Column(String)
    opt_momentum = Column(Float)
    opt_offset = Column(Float)
    opt_decay = Column(Float)
    opt_step_rate = Column(Float)

    for _prefix in _model_prefixes:
        exec("""
{0}best_val_loss = Column(Float)
{0}best_parameters = Column(PickleType, nullable=True)
{0}train_loss = Column(PickleType)
{0}val_loss = Column(PickleType)
{0}epochs = Column(Integer)
{0}init_params = Column(PickleType, nullable=True)
{0}val_error_rate = Column(Float)""".format(_prefix))
    def __init__(self, x_true):
        """
        Parameters
        ----------
        datasetname : str
        """
        self.x_true = x_true
        self.max_epochs_without_improvement = 30
        self.logP_average_n = 3  # TODO random.choice([1,10])
        self.errorrate_average_n = 10
        self.init_results()

    def init_results(self):

        # extra for being able to reset results for loaded hyperparameters
        for prefix in _model_prefixes:
            setattr(self, prefix + "best_parameters", None)
            setattr(self, prefix + "best_val_loss", inf)
            setattr(self, prefix + "train_loss", [])
            setattr(self, prefix + "val_loss", [])
            setattr(self, prefix + "best_epoch", 0)
            setattr(self, prefix + "init_params", None)
            setattr(self, prefix + "val_error_rate", inf)
            
    def __repr__(self):
        return "hyper %i" % hash(self)

            

# Collect models and find best ones

In [9]:
best_hyper = eva.get_best_hyper(["toy_windows", "toy_linux"], Hyper, model_prefixes,
                                n_best=2, test_suffix=["best_val_loss"])

# Compute Results on Test-set

In [10]:
from sklearn import cross_validation

In [11]:
dim=1
error_func = experiment_util.RMSE
sampler = experiment_toy_models.toy_likelihood(dim=dim).function()

def data_gen(hyper):
    hyper.dim = dim
    x_true = np.array([hyper.x_true]*hyper.dim, dtype=theano.config.floatX)
    _Z = np.array([sampler(x_true) for n in range(1000)], dtype=theano.config.floatX)
    Z, TZ = cross_validation.train_test_split(_Z, test_size=0.1)  # 10% test used in paper
    Z, VZ = cross_validation.train_test_split(Z, test_size=0.1)  # 20% validation used in paper
    return None, Z, None, VZ, None, TZ  # None represents X data

In [12]:
test_results = eva.compute_test_results(best_hyper, data_gen, error_func, experiment_toy_models, n_trials=3)

batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 

/home/stephan/GitProjects/climin/climin/util.py:151: UserWarning: Argument named f is not expected by <class 'climin.rmsprop.RmsProp'>
  % (i, klass))
/home/stephan/GitProjects/climin/climin/util.py:151: UserWarning: Argument named f is not expected by <class 'climin.adam.Adam'>
  % (i, klass))



1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37
batch_precompile
batch_precompile
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 

/home/stephan/GitProjects/climin/climin/util.py:151: UserWarning: Argument named f is not expected by <class 'climin.adadelta.Adadelta'>
  % (i, klass))


# Sample the approximate posterior distribution for evaluation

In [ ]:
best_hyper_samples = eva.sample_best_hyper(best_hyper)

## Histogram (only possible for toy dataset)

In [ ]:
for i in range(n_best):
    ln = len(best_hyper_samples['best_val_loss'].keys())
    fig, axs = plt.subplots(nrows=ln, ncols=len(n_normflows), sharex=True, sharey=True)

    axs_iter = axs.flat
    for name in model_prefixes:
        for nn in n_normflows:
            if name == "baselinedet":
                continue
            samples = best_hyper_samples['best_val_loss'][name][nn][i]
            ax = next(axs_iter)
            ax.hist(samples, bins="auto") #50
            ax.tick_params(axis='both', which='major', labelsize=5)
            ax.tick_params(axis='both', which='minor', labelsize=5)
    #         ax.set_ylabel("frequency")
    #         ax.set_xlabel("parameter")
            ax.set_title("%s, %i" % (name, nn), fontsize=9)
    plt.xlim([-0.6, 1.1])
    fig.tight_layout()

## number of modes / histogram of modes

In [ ]:
nr_modes = eva.get_nr_modes(best_hyper_samples)
nr_modes

## KL divergence

This must be computed on histograms, as we need an approximation for the densities/pms

In [ ]:
best_kl = eva.compute_kl()

In [ ]:
for nn in n_normflows:
    means = np.mean(best_kl['best_val_loss'][nn], axis=0)
    variances = np.var(best_kl['best_val_loss'][nn], axis=0)

    fig, axs = plt.subplots(ncols=2)
    h = axs[0].imshow(means, interpolation="None", cmap=plt.cm.RdBu_r, norm=Centre(0))
    plt.colorbar(h)

    h = axs[1].imshow(variances, interpolation="None", cmap=plt.cm.gray_r, norm=LogNorm())
    plt.colorbar(h)
    plt.title("nn = %i" %nn)

In [ ]:
variances